# Test Variational Inference algorithms for Wold processes

In this notebook, we test the various variational inference (VI) algorithms implemented for the multivariate Wold process. Namely:

* `WoldModelVariational`: VI with parameters $\alpha$ and $\beta$

with the **new updates** involving solving a set of 2d^2 1-dimensional equations.

---

Import the libraries of interest for this notebook

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

import itertools
import torch
import numpy as np

# Set numpy print format
np.set_printoptions(precision=2, floatmode='fixed', sign=' ')

import tsvar
import experiments_utils

Set cells width

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

---

## Generate toy example dataset


### V1. Small 2-dimensional toy setting

### V2. Larger setting

In [4]:
from experiments_utils import generate_parameters

dim = 50
max_jumps = 10e3 * dim

param_dict = generate_parameters(dim=dim, base_range=[0.05, 0.05], adj_range=[0.1, 0.2], seed=11)

mu = torch.tensor(param_dict['baseline'])
beta = torch.tensor(param_dict['beta'])
alpha = torch.tensor(param_dict['adjacency'])

print('Baseline:')
print(mu.numpy().round(2))
print('Alpha:')
print(alpha.numpy().round(2))
print('Beta:')
print(beta.numpy().round(2))

Baseline:
[ 0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05
  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05
  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05
  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05  0.05
  0.05  0.05]
Alpha:
[[ 0.00  0.00  0.00 ...  0.00  0.00  0.00]
 [ 0.00  0.11  0.00 ...  0.12  0.00  0.18]
 [ 0.00  0.00  0.00 ...  0.00  0.00  0.00]
 ...
 [ 0.00  0.00  0.00 ...  0.20  0.00  0.00]
 [ 0.00  0.00  0.00 ...  0.00  0.00  0.00]
 [ 0.00  0.00  0.18 ...  0.00  0.00  0.00]]
Beta:
[[ 0.62  0.36  0.17 ...  0.57  0.66  0.58]
 [ 0.32  0.22  0.84 ...  0.14  0.14  0.93]
 [ 0.96  0.97  0.36 ...  0.30  0.03  0.42]
 ...
 [ 0.38  0.67  0.80 ...  0.68  0.54  0.28]
 [ 0.15  0.95  0.82 ...  0.74  0.70  0.71]
 [ 0.32  0.84  0.67 ...  0.42  0.69  0.92]]


### Simulate a realization

---

## Test Mean-Field VI with Variable $\{\beta\}$s using `WoldModelVariational`

Define the parameters of the prior.

---

## Cython code

In [7]:
import pickle
with open('test_model.pk', 'rb') as f:
    model = pickle.load(f)

In [ ]:
from tsvar.models.cython import cython_wold_var_AltDeltaInit

cython_wold_var_AltDeltaInit._update_z(as_po=model._as_po, ar_po=model._ar_po, 
                                       bs_po=model._bs_po, br_po=model._br_po, 
                                       dt_ik=model.dt_ik, 
                                       delta_ikj=model.delta_ikj);

In [9]:
import tsvar.models._wold_var_AltDeltaInit 
from tsvar.models.cython import cython_wold_var_AltDeltaInit

# compile jit
tsvar.models._wold_var_AltDeltaInit._update_z(as_po=model._as_po, ar_po=model._ar_po, 
                                    bs_po=model._bs_po, br_po=model._br_po, 
                                    dt_ik=model.dt_ik, 
                                    delta_ikj=model.delta_ikj);

In [12]:
%%timeit

tsvar.models._wold_var_AltDeltaInit._update_z(as_po=model._as_po, ar_po=model._ar_po, 
                                    bs_po=model._bs_po, br_po=model._br_po, 
                                    dt_ik=model.dt_ik, 
                                    delta_ikj=model.delta_ikj);

809 ms ± 35.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit

cython_wold_var_AltDeltaInit._update_z(as_po=model._as_po, ar_po=model._ar_po, 
                                       bs_po=model._bs_po, br_po=model._br_po, 
                                       dt_ik=model.dt_ik, 
                                       delta_ikj=model.delta_ikj);

1.33 s ± 580 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
